#  NSRDB and WTK Data Slicing

This notebook demonstrates data slicing for the National Renewable Energy Laboratory (NREL) National Solar Radiation Database (NSRDB) data. The data is provided from Amazon Web Services using the HDF Group's Highly Scalable Data Service (HSDS). These slicing methods would also work with the WIND Toolkit data available via HSDS at /nrel/wtk/

For this to work you must first install h5pyd:

```
pip install --user h5pyd
```

Next you'll need to configure HSDS:

```
hsconfigure
```

and enter at the prompt:

```
hs_endpoint = https://developer.nrel.gov/api/hsds
hs_username = None
hs_password = None
hs_api_key = 3K3JQbjZmWctY0xmIfSYvYgtIcM3CN0cb1Y2w9bf
```

*The example API key here is for demonstation and is rate-limited per IP. To get your own API key, visit https://developer.nrel.gov/signup/*

You can also add the above contents to a configuration file at ~/.hscfg

This example will also be using NREL's Resource Extraction Tools package [`NREL-rex`](https://github.com/nrel/rex) which can be installed with pip:
```
pip install NREL-rex
```
or with conda:
```
conda install nrel-rex
```

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from rex import NSRDBX, WindX

# NSRSBX

In [ ]:
# NSRDBX is a wrapper on h5py and h5pyd that simplifies extraction of NSRDB data
# It automatically converts the meta data to a pandas DataFrame
# The time_index to a pandas DatatimeIndex
# And unscales NSRDB datasets
# NOTE: the syntax for extracting datasets is different from h5py/h5pyd!!
nsrdb_file = '/nrel/nsrdb/v3/nsrdb_2010.h5'
with NSRDBX(nsrdb_file, hsds=True) as f:
    meta = f.meta
    time_index = f.time_index
    dni = f['dni', :, ::1000]

display(meta.head())
diplay(time_index)
dni.shape

In [10]:
# NSRDBX also allows easy extraction of the nearest site to a desired (lat, lon) location:
coords = (39.741931, -105.169891)
with NSRDBX(nsrdb_file, hsds=True) as f:
    dni_df = f.get_lat_lon_df('dni', coords)

sns.scatterplot(x="time_index", y="dni", data=dni_df)

In [ ]:
# NSRDBX can also filter on differnt meta data columns
state='Colorado'
with NSRDBX(nsrdb_file, hsds=True) as f:
    state_dni = f.get_region_df('dni', state, region_col='state')